# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
df = pd.read_csv("../output_data/weather_output.csv")
weather_df = df.drop(columns=["Unnamed: 0"])
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ponta do sol,75,PT,1601342873,82,32.67,-17.10,20.00,6.20
1,busselton,100,AU,1601342873,94,-33.65,115.33,16.11,2.68
2,albany,0,US,1601342874,84,42.60,-73.97,22.22,3.23
3,cape town,9,ZA,1601342874,93,-33.93,18.42,12.22,2.10
4,zhigansk,100,RU,1601342875,95,66.77,123.37,1.54,11.27


In [14]:
# Transform from celsius to fahrenheit
Max_Temp_F = [(x*1.8) + 32 for x in weather_df["Max Temp"]]


In [15]:
# Add to DataFrame as a new column
weather_df["Max Temp (F)"] = Max_Temp_F
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Max Temp (F)
0,ponta do sol,75,PT,1601342873,82,32.67,-17.10,20.00,6.20,68.000
1,busselton,100,AU,1601342873,94,-33.65,115.33,16.11,2.68,60.998
2,albany,0,US,1601342874,84,42.60,-73.97,22.22,3.23,71.996
3,cape town,9,ZA,1601342874,93,-33.93,18.42,12.22,2.10,53.996
4,zhigansk,100,RU,1601342875,95,66.77,123.37,1.54,11.27,34.772


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Humidity Heatmap

weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)
locations = weather_df[["Lat", "Lng"]].astype(float)

# Add heatmap figure to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
sum_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70
sum_weather_df = sum_weather_df.loc[(sum_weather_df["Max Temp (F)"] < 80) & (sum_weather_df["Max Temp (F)"] > 70)]

# Wind speed less than 10 mph
sum_weather_df = sum_weather_df.loc[sum_weather_df["Wind Speed"] < 10]

# Zero cloudiness
sum_weather_df = sum_weather_df.loc[sum_weather_df["Cloudiness"] == 0]

sum_weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Max Temp (F)
2,albany,0,US,1601342874,84,42.60,-73.97,22.22,3.23,71.996
33,mackay,0,AU,1601342884,47,-21.15,149.20,26.00,7.70,78.800
46,karpathos,0,GR,1601342888,94,35.51,27.21,24.00,1.50,75.200
47,mangrol,0,IN,1601342889,78,21.12,70.12,26.10,3.09,78.980
64,mandiana,0,GN,1601342894,79,10.63,-8.68,23.90,0.99,75.020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Hotel Map

hotel_df = sum_weather_df

# Set parameters to search hotels with 5000 meters
params = {
    "key": g_key,
    "radius": 5000,
    "keyword": "hotel",
    }

for index, row in hotel_df.iterrows():
    # Get lat, lng from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Create url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(url, params=params).json()
    
    # Extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError  ):
        print("Result not found... Skipping.")
        
    print("------------")


Retrieving Results for Index 2: albany.
Result not found... Skipping.
------------
Retrieving Results for Index 33: mackay.
Closest hotel is Mantra Mackay.
------------
Retrieving Results for Index 46: karpathos.
Closest hotel is Konstantinos Palace.
------------
Retrieving Results for Index 47: mangrol.
Closest hotel is Kaveri Hotel.
------------
Retrieving Results for Index 64: mandiana.
Result not found... Skipping.
------------
Retrieving Results for Index 76: raiwind.
Result not found... Skipping.
------------
Retrieving Results for Index 80: atuona.
Closest hotel is Hôtel Hiva Oa Hanakee Lodge.
------------
Retrieving Results for Index 107: vernon.
Closest hotel is Prestige Vernon Lodge and Conference Centre.
------------
Retrieving Results for Index 180: alta floresta.
Closest hotel is Floresta Amazonica Hotel.
------------
Retrieving Results for Index 186: caravelas.
Closest hotel is Pousada Encanto Abrolhos.
------------
Retrieving Results for Index 197: deori.
Result not foun

In [19]:
# Store hotel result into a DataFrame
hotel_df.to_csv("../output_data/hotel_output.csv")
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Max Temp (F),Hotel Name
2,albany,0,US,1601342874,84,42.60,-73.97,22.22,3.23,71.996,NaN
33,mackay,0,AU,1601342884,47,-21.15,149.20,26.00,7.70,78.800,Mantra Mackay
46,karpathos,0,GR,1601342888,94,35.51,27.21,24.00,1.50,75.200,Konstantinos Palace
47,mangrol,0,IN,1601342889,78,21.12,70.12,26.10,3.09,78.980,Kaveri Hotel
64,mandiana,0,GN,1601342894,79,10.63,-8.68,23.90,0.99,75.020,NaN


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in sum_weather_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map

# Add marker layer of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
symbol_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)
fig.add_layer(symbol_layer)
# Display Map
fig


Figure(layout=FigureLayout(height='420px'))